In [ ]:
#r "nuget:Microsoft.ML.TimeSeries"
#r "nuget:Microsoft.Data.Analysis"

using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.Transforms.TimeSeries;
using XPlot.Plotly;

In [ ]:
DataFrame usage = DataFrame.LoadCsv(@"usage.csv");

// split the data into 2
DataFrame coreUsage = new DataFrame(usage.Columns[0], usage.Columns[1]);
DataFrame netUsage = new DataFrame(usage.Columns[0], usage.Columns[2]);

coreUsage.Columns[1].SetName("Usage");
netUsage.Columns[1].SetName("Usage");

In [ ]:
var coreSeries = new Graph.Scattergl
{
    name = ".NET Core",
    y = coreUsage.Columns[1],
    x = coreUsage.Columns[0],
};

var netSeries = new Graph.Scattergl
{
    name = ".NET Framework",
    y = netUsage.Columns[1],
    x = netUsage.Columns[0],
};

var chart = Chart.Plot(new[] {coreSeries, netSeries});
chart.WithTitle(".NET Core vs Framework Usage Current");
display(chart);

In [ ]:
/// <summary>
/// This is the input of the scored time series model, the past data.
/// </summary>
public class UsageData
{
    public float Usage { get; set; }
}

/// <summary>
/// This is the output of the scored time series model, the prediction.
/// </summary>
public class UsagePrediction
{
    public float[] ForecastedUsage { get; set; }

    public float[] ConfidenceLowerBound { get; set; }

    public float[] ConfidenceUpperBound { get; set; }
}

var numSeriesDataPoints = (int)usage.Rows.Count;

var mlContext = new MLContext();
IEstimator<ITransformer> forecastEstimator = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: nameof(UsagePrediction.ForecastedUsage), 
    inputColumnName: nameof(UsageData.Usage), // This is the column being forecasted.
    windowSize: 6, // Window size is set to the time period represented in the product data cycle; our product cycle is based on 12 months, so this is set to a factor of 12, e.g. 3.
    seriesLength: numSeriesDataPoints, // This parameter specifies the number of data points that are used when performing a forecast.
    trainSize: numSeriesDataPoints, // This parameter specifies the total number of data points in the input time series, starting from the beginning.
    horizon: 25, // Indicates the number of values to forecast; 2 indicates that the next 2 months of product units will be forecasted.
    confidenceLevel: 0.95f, // Indicates the likelihood the real observed value will fall within the specified interval bounds.
    confidenceLowerBoundColumn: nameof(UsagePrediction.ConfidenceLowerBound), //This is the name of the column that will be used to store the lower interval bound for each forecasted value.
    confidenceUpperBoundColumn: nameof(UsagePrediction.ConfidenceUpperBound)); //This is the name of the column that will be used to store the upper interval bound for each forecasted value.

// Fit the forecasting model to the specified product's data series.
ITransformer coreUsageForecastTransformer = forecastEstimator.Fit(coreUsage);
var coreForecastEngine = coreUsageForecastTransformer.CreateTimeSeriesEngine<UsageData, UsagePrediction>(mlContext);
var coreUsageEstimation = coreForecastEngine.Predict();

ITransformer netUsageForecastTransformer = forecastEstimator.Fit(netUsage);
var netForecastEngine = netUsageForecastTransformer.CreateTimeSeriesEngine<UsageData, UsagePrediction>(mlContext);
var netUsageEstimation = netForecastEngine.Predict();

In [ ]:
var coreSeries = new Graph.Scattergl
{
    name = ".NET Core",
    y = coreUsageEstimation.ForecastedUsage,
    x = Enumerable.Range(0, 24)
};

var netSeries = new Graph.Scattergl
{
    name = ".NET Framework",
    y = netUsageEstimation.ForecastedUsage,
    x = Enumerable.Range(0, 24)
};

var chart = Chart.Plot(new[] {coreSeries, netSeries});
chart.WithTitle(".NET Core vs Framework Usage Forecast");
display(chart);